# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Udachny,66.4167,112.4000,-16.58,93,100,2.55,RU,1731195404
1,1,Sretensk,52.2453,117.7093,-13.89,89,16,1.68,RU,1731195406
2,2,Wailua homesteads,22.0669,-159.3780,27.44,79,75,7.20,US,1731195408
3,3,Edinburgh of the seven seas,-37.0676,-12.3116,15.39,91,100,3.75,SH,1731195431
4,4,Calvinia,-31.4707,19.7760,15.38,46,2,2.68,ZA,1731195470


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size="Humidity",
    tiles = "OSM",
    color="City",
    alpha=0.5
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [33]:
# Narrow down cities that fit criteria and drop any results with null values
new_city_df = city_data_df.loc[(city_data_df["Max Temp"] <= 25) & (city_data_df["Max Temp"] >= 15) & (city_data_df["Cloudiness"] <= 50), :]

# Drop any rows with null values
new_city_df = new_city_df.dropna(how='any')

# Display sample data
new_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,Calvinia,-31.4707,19.7760,15.38,46,2,2.68,ZA,1731195470
9,9,Al jawf,29.5000,38.7500,18.15,63,44,3.04,SA,1731195482
14,14,Cairo,30.0626,31.2497,20.42,77,20,0.00,EG,1731195003
20,20,Bani walid,31.7566,13.9942,17.35,73,1,1.77,LY,1731195503
25,25,Maputo,-25.9653,32.5892,23.82,83,20,0.51,MZ,1731195377
...,...,...,...,...,...,...,...,...,...,...
576,576,Jwaneng,-24.6004,24.7303,18.76,78,45,3.17,BW,1731196560
582,582,Molina,-35.1167,-71.2833,20.76,58,0,4.12,CL,1731196571
599,599,Pigeon forge,35.7884,-83.5543,19.99,77,20,0.00,US,1731196602
601,601,Paulden,34.8856,-112.4682,18.10,27,0,0.70,US,1731196350


### Step 3: Create a new DataFrame called `hotel_df`.

In [34]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = new_city_df[["City", "Country", "Lat", "Lng", "Humidity"]]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\HP\AppData\Local\Temp\ipykernel_32928\2981621922.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
4,Calvinia,ZA,-31.4707,19.7760,46,
9,Al jawf,SA,29.5000,38.7500,63,
14,Cairo,EG,30.0626,31.2497,77,
20,Bani walid,LY,31.7566,13.9942,73,
25,Maputo,MZ,-25.9653,32.5892,83,
...,...,...,...,...,...,...
576,Jwaneng,BW,-24.6004,24.7303,78,
582,Molina,CL,-35.1167,-71.2833,58,
599,Pigeon forge,US,35.7884,-83.5543,77,
601,Paulden,US,34.8856,-112.4682,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [35]:
# Set parameters to search for a hotel
radius = 100000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lng"]
    lng =  row["Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        # Convert the API response to JSON format
        name_address = response.json()
        print(f"Response for {row['City']}:", name_address)

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    else:
        print(f"Error fetching data for {row['City']}: {response.status_code}")
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Response for Calvinia: {'type': 'FeatureCollection', 'features': []}
Calvinia - nearest hotel: No hotel found
Response for Al jawf: {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Uşak Polis Evi', 'country': 'Turkey', 'country_code': 'tr', 'state': 'Aegean Region', 'county': 'Uşak', 'city': 'Uşak', 'postcode': '64400', 'suburb': 'Köme Mahallesi', 'street': 'Nuri Şeker Caddesi', 'lon': 29.4093164, 'lat': 38.6796582, 'formatted': 'Uşak Polis Evi, Nuri Şeker Caddesi, 64400 Uşak, Turkey', 'address_line1': 'Uşak Polis Evi', 'address_line2': 'Nuri Şeker Caddesi, 64400 Uşak, Turkey', 'categories': ['accommodation', 'accommodation.hotel'], 'details': ['details'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Uşak Polis Evi', 'osm_id': 4864432306, 'tourism': 'hotel', 'website': 'https://

,City,Country,Lat,Lng,Humidity,Hotel Name
4,Calvinia,ZA,-31.4707,19.7760,46,No hotel found
9,Al jawf,SA,29.5000,38.7500,63,Uşak Polis Evi
14,Cairo,EG,30.0626,31.2497,77,No hotel found
20,Bani walid,LY,31.7566,13.9942,73,No hotel found
25,Maputo,MZ,-25.9653,32.5892,83,No hotel found
...,...,...,...,...,...,...
576,Jwaneng,BW,-24.6004,24.7303,78,No hotel found
582,Molina,CL,-35.1167,-71.2833,58,No hotel found
599,Pigeon forge,US,35.7884,-83.5543,77,No hotel found
601,Paulden,US,34.8856,-112.4682,27,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size="Humidity",
    tiles = "OSM",
    color="City",
    alpha=0.5,
    hover_cols=["Hotel Name", "Country"]
)


# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)